# __Frame extraction__
Once a suitable MD has been determined, frames that describe a precatalytic structure are identified through a set of criteria and are then extracted for QM/MM optimization and possible scan.

This procedure reuses code from `analyze.ipynb` from the MD analysis step.

In [27]:
# IMPORT SECTION
# general 
import os
import sys
from numpy import average, std
import numpy as np
np.set_printoptions(suppress=True, threshold=sys.maxsize)

# md analysis

from EMDA import EMDA, __version__
from EMDA import frame_extractor
print("EMDA version is:", __version__)

# plotting
import matplotlib.pyplot as plt
plt.style.use('default')
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('lines', lw=1, color='b')
rc('legend', loc='best')
plt.rcParams["legend.fancybox"] = False
plt.rcParams["legend.edgecolor"] = 'black'
plt.rcParams['legend.borderpad'] = 0.25
plt.rcParams['legend.fontsize'] = 11
plt.rcParams.update({'pgf.preamble': r'\usepackage{amsmath}'})

EMDA version is: 1.0.0a5


In [28]:
# var definition
md_base_dir = '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction'                       
parent_dir = 'md_run'   
protein_ID = 'hLOX15'       
lig_ID = '5-HETE'               
load_results = True                                        

selection = 13
dist_criterium = 3.0
dih_criterium = 5.0


lig_ID_list = ['AA', '5-HpETE', '5-HETE']
if lig_ID not in lig_ID_list:
    raise ValueError(f'Introduced lig_ID {lig_ID} is not among the supported ligands: {lig_ID_list}')

# topology
topologies_1 = [i for i in os.listdir(os.path.join(md_base_dir, parent_dir)) if i.endswith('.prmtop')]
top = os.path.join(md_base_dir, parent_dir, topologies_1[0])
if len(topologies_1) > 1:
    raise ValueError('bruh. More than one topology files? Check md source directory.')
print(f'loading prmtop\t{top}')

# production trajectory
prod_dir_1 = os.path.join(md_base_dir, parent_dir, 'prod/out')                    # locate the production traj
prod_files = [int(x.strip('.nc').strip('prod_')) for x in os.listdir(prod_dir_1) if '.nc' in x]
traj_1 = [os.path.join(prod_dir_1, f'prod_{i}.nc')  for i in prod_files]      # assume constant length 250 ns, 25 
print(np.array(traj_1))

# load base MD (r=1)
emda = EMDA()
emda.load_variant(
    parameters=top,
    trajectory=traj_1,
    variant_name=protein_ID
)

total_time_simulated = 10 * len(traj_1)
print(f'total time simulated: {total_time_simulated} ns')

loading prmtop	/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/h15LOX_5-HETE_343_12_mod_norelaxed_4amber_solv.prmtop
['/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_1.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_2.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_3.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_4.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_5.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_6.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_7.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_8.nc'
 '/Volumes/white_HDD/qmmm/QMMM_5HETE_343/1_frames/frame_extraction/md_run/prod/out/prod_9.nc'
 '/Volumes/wh

/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/topology/TOPParser.py:342: UserWarning: Unknown ATOMIC_NUMBER value found for some atoms, these have been given an empty element record. If needed these can be guessed using universe.guess_TopologyAttrs(to_guess=['elements']).
  warnings.warn(msg)


hLOX15 variant has been loaded!
total time simulated: 500 ns


In [29]:
print('performing atom selections')
if lig_ID == lig_ID_list[0]:

    #===============#
    #       AA      #
    #===============#

    print(f'selecting atoms for {lig_ID_list[0]}')

    # LIGAND
    emda.select('LIG', '665', sel_type='res_num')

    emda.select('H10A', 'name H10A and resid 665')
    #emda.select('H10B', 'name H10B and resid 665')

    emda.select('H13A', 'name H13A and resid 665')
    #emda.select('H13B', 'name H13B and resid 665')

    # planarity dihedrals
    # C10 planarity
    emda.select('p10_A', 'name C12 and resid 665')      # A i B formen el cis double bond
    emda.select('p10_B', 'name C11 and resid 665')
    emda.select('p10_C', 'name C10 and resid 665')      # C conté H10A i H10B
    emda.select('p10_D', 'name C9 and resid 665')       # D és el carboni més proxim al COO

    # C13 planarity
    emda.select('p13_A', 'name C11 and resid 665')      # A i B formen el cis double bond
    emda.select('p13_B', 'name C12 and resid 665')
    emda.select('p13_C', 'name C13 and resid 665')      # C conté H13A i H13B
    emda.select('p13_D', 'name C14 and resid 665')      # D és el carboni més llunyà al COO

    # PROTEIN hLOX15
    # OH1 OH cofactor :664
    emda.select('OH1', 10552, sel_type='at_num') 

    print(emda.selections)


elif lig_ID == lig_ID_list[1]:

    #====================#
    #       5-HpETE      #
    #====================#
    
    print(f'selecting atoms for {lig_ID_list[1]}')

    # LIGAND
    emda.select('LIG', '665', sel_type='res_num')
    
    emda.select('H10A', 'name H19 and resid 665')
    #emda.select('H10B', 'name H18 and resid 665')

    emda.select('H13A', 'name H15 and resid 665')
    #emda.select('H13B', 'name H14 and resid 665')

    # planarity dihedrals
    # C10 planarity
    emda.select('p10_A', 'name C9 and resid 665')       # A i B formen el cis double bond
    emda.select('p10_B', 'name C10 and resid 665')    
    emda.select('p10_C', 'name C11 and resid 665')      # C conté H10A i H10B
    emda.select('p10_D', 'name C12 and resid 665')      # D és el carboni més proxim al COO

    # C13 planarity
    emda.select('p13_A', 'name C10 and resid 665')      # A i B formen el cis double bond
    emda.select('p13_B', 'name C9 and resid 665')
    emda.select('p13_C', 'name C8 and resid 665')       # C conté H13A i H13B
    emda.select('p13_D', 'name C7 and resid 665')       # D és el carboni més llunyà al COO

    # PROTEIN hLOX15
    # OH1 OH cofactor :664
    #                   O
    emda.select('OH1', 10552, sel_type='at_num') 

    print(emda.selections)


elif lig_ID == lig_ID_list[2]:

    #===================#
    #       5-HETE      #
    #===================#

    print(f'selecting atoms for {lig_ID_list[2]}')

    # ligand atoms
    emda.select('LIG', '665', sel_type='res_num')

    emda.select('H10A', 'name H19 and resid 665')
    #emda.select('H10B', 'name H18 and resid 665')

    emda.select('H13A', 'name H15 and resid 665')
    #emda.select('H13B', 'name H14 and resid 665')

    # planarity dihedrals
    # C10 planarity
    emda.select('p10_A', 'name C9 and resid 665')       # A i B formen el cis double bond
    emda.select('p10_B', 'name C10 and resid 665')    
    emda.select('p10_C', 'name C11 and resid 665')      # C conté H10A i H10B
    emda.select('p10_D', 'name C12 and resid 665')      # D és el carboni més proxim al COO

    # C13 planarity
    emda.select('p13_A', 'name C10 and resid 665')      # A i B formen el cis double bond
    emda.select('p13_B', 'name C9 and resid 665')
    emda.select('p13_C', 'name C8 and resid 665')       # C conté H13A i H13B
    emda.select('p13_D', 'name C7 and resid 665')       # D és el carboni més llunyà al COO

    # PROTEIN hLOX15
    # OH1 OH cofactor :664
    #                   O
    emda.select('OH1', 10552, sel_type='at_num') 

    print(emda.selections)

else:
    raise ValueError('Something went wrong!')

performing atom selections
selecting atoms for 5-HETE
{'LIG': 'resid 665', 'H10A': 'name H19 and resid 665', 'H13A': 'name H15 and resid 665', 'p10_A': 'name C9 and resid 665', 'p10_B': 'name C10 and resid 665', 'p10_C': 'name C11 and resid 665', 'p10_D': 'name C12 and resid 665', 'p13_A': 'name C10 and resid 665', 'p13_B': 'name C9 and resid 665', 'p13_C': 'name C8 and resid 665', 'p13_D': 'name C7 and resid 665', 'OH1': 'bynum 10552'}


In [30]:
emda.add_distance('distance H10A -- O OH1', 'OH1', 'H10A')
#emda.add_distance('distance H10B -- O OH1', 'OH1', 'H10B')

emda.add_distance('distance H13A -- O OH1', 'OH1', 'H13A')
#emda.add_distance('distance H13B -- O OH1', 'OH1', 'H13B')

emda.add_dihedral('dihedral C12--C11--C10--C9 (planarity around C10)', 'p10_A', 'p10_B', 'p10_C', 'p10_D', domain=360)     # around C10
emda.add_dihedral('dihedral C11--C12--C13--C14 (planarity around C13)', 'p13_A', 'p13_B', 'p13_C', 'p13_D', domain=360)    # around C13

print('selected measurements:\n')
print(emda.measures)

selected measurements:

{'distance H10A -- O OH1': Measure dataclass with:
	Name:   distance H10A -- O OH1
	Type:   distance
	Sel:    ['OH1', 'H10A']
	Status: 
		hLOX15, R1: Not calculated
, 'distance H13A -- O OH1': Measure dataclass with:
	Name:   distance H13A -- O OH1
	Type:   distance
	Sel:    ['OH1', 'H13A']
	Status: 
		hLOX15, R1: Not calculated
, 'dihedral C12--C11--C10--C9 (planarity around C10)': Measure dataclass with:
	Name:   dihedral C12--C11--C10--C9 (planarity around C10)
	Type:   dihedral
	Sel:    ['p10_A', 'p10_B', 'p10_C', 'p10_D']
	Status: 
		hLOX15, R1: Not calculated
, 'dihedral C11--C12--C13--C14 (planarity around C13)': Measure dataclass with:
	Name:   dihedral C11--C12--C13--C14 (planarity around C13)
	Type:   dihedral
	Sel:    ['p13_A', 'p13_B', 'p13_C', 'p13_D']
	Status: 
		hLOX15, R1: Not calculated
}


In [31]:
frames_dir = os.path.join(md_base_dir, 'precat')
if not load_results:
    # compute the metrics
    emda.run()
    
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    # save the analysis
    emda.save(os.path.join(frames_dir, f'{protein_ID}_{lig_ID}_frames'))    

else: 
    # load precomputed analysis 
    emda.load(os.path.join(frames_dir, f'{protein_ID}_{lig_ID}_frames.pkl'))

print(f'loaded systems:\t{emda.universe}')
print(f'\ntotal measures conducted:\n{emda.measures}')

EMDA measures have been loaded!
EMDA selections have been loaded!
loaded systems:	{'hLOX15': {'R1': <Universe with 104548 atoms>}}

total measures conducted:
{'distance H10A -- O OH1': Measure dataclass with:
	Name:   distance H10A -- O OH1
	Type:   distance
	Sel:    ['OH1', 'H10A']
	Status: 
		hLOX15, R1: Calculated
, 'distance H13A -- O OH1': Measure dataclass with:
	Name:   distance H13A -- O OH1
	Type:   distance
	Sel:    ['OH1', 'H13A']
	Status: 
		hLOX15, R1: Calculated
, 'dihedral C12--C11--C10--C9 (planarity around C10)': Measure dataclass with:
	Name:   dihedral C12--C11--C10--C9 (planarity around C10)
	Type:   dihedral
	Sel:    ['p10_A', 'p10_B', 'p10_C', 'p10_D']
	Status: 
		hLOX15, R1: Calculated
, 'dihedral C11--C12--C13--C14 (planarity around C13)': Measure dataclass with:
	Name:   dihedral C11--C12--C13--C14 (planarity around C13)
	Type:   dihedral
	Sel:    ['p13_A', 'p13_B', 'p13_C', 'p13_D']
	Status: 
		hLOX15, R1: Calculated
}


In [32]:
# perform the analyses
if selection == 10:
    print('analyzing C10 dist and dihedral')
    emda.analyse_value('dihedral_criterion', 'dihedral C12--C11--C10--C9 (planarity around C10)', mode='thres', val1=180-dih_criterium, val2=180+dih_criterium)
    emda.analyse_value('distance_criterion', 'distance H10A -- O OH1', mode='thres', val1=dist_criterium)
    # obtain the results
    criterion_1 = list(emda.analyses['dihedral_criterion'].result.values())[0]['R1']
    criterion_2 = list(emda.analyses['distance_criterion'].result.values())[0]['R1']


elif selection == 13:
    print('analyzing C13 dist and dihedral')
    
    emda.analyse_value('dihedral_criterion', 'dihedral C11--C12--C13--C14 (planarity around C13)', mode='thres', val1=180-dih_criterium, val2=180+dih_criterium)
    emda.analyse_value('distance_criterion', 'distance H13A -- O OH1', mode='thres', val1=dist_criterium)
    print(emda.measures['dihedral C11--C12--C13--C14 (planarity around C13)'].result.values())

    # obtain the results
    criterion_1 = list(emda.analyses['dihedral_criterion'].result.values())[0]['R1']
    criterion_2 = list(emda.analyses['distance_criterion'].result.values())[0]['R1']

else:
    raise ValueError('ep! Analysis step exploded!')

# obtain the results
criterion_1 = list(emda.analyses['dihedral_criterion'].result.values())[0]['R1']
criterion_2 = list(emda.analyses['distance_criterion'].result.values())[0]['R1']

# AND-type restriction
criteria = list(np.array([criterion_1, criterion_2]).min(axis=0))

# printing
print(f'Dihedral criterion\n{criterion_1}\n')
print(f'Distance criterion\n{criterion_2}\n')
print(f'Both ({criteria.count(True)} frames)\n{criteria}\n')

print(f'Frames that satisfy the dihedral criterion:\t{(criterion_1.count(True)*100/(1000 * len(traj_1))):.2f}\t%')
print(f'Frames that satisfy the distance criterion:\t{(criterion_2.count(True)*100/(1000 * len(traj_1))):.2f}\t%')
print(f'Frames that satisfy both criteria:\t\t{(criteria.count(True)*100/(1000 * len(traj_1))):.2f}\t%')

analyzing C13 dist and dihedral
dict_values([{'R1': [128.39860718480293, 135.5705344573687, 125.79082406333146, 124.80906025263926, 131.47712554015084, 122.68221752578614, 111.90299683581804, 125.78466552849994, 129.0922483085476, 123.57265608763606, 119.29958447761345, 121.40364381651591, 124.21966368259302, 121.0539277876187, 127.07001798306857, 113.23428368654322, 120.97236082788146, 123.61445188720415, 133.53783200358578, 118.69241115630547, 117.9054535207232, 130.59693613192934, 105.36395091458303, 117.55318108407909, 118.26568357697852, 117.49315696999457, 128.24585813676373, 126.56712193237661, 126.59397940713438, 118.4007070794961, 130.83741268439826, 104.10064677724802, 121.40006767773127, 120.7814478270742, 108.07701322383775, 114.32916404112154, 109.20461637079711, 128.3020500351908, 118.19750168007732, 114.79209035420769, 131.907412376141, 123.38884385566195, 107.63532521228251, 126.52421275943834, 114.61694074445955, 119.56784363850312, 129.38375635842584, 119.433817267823

In [33]:
# generate new analysis attribute with the combined results (EMDA is lacking lmao)
emda.analyses['combined_criteria'] = emda.analyses['distance_criterion']
emda.analyses['combined_criteria'].result = {protein_ID : {'R1' : criteria}}


# TODO
# randomly select a set of frames that fulfill the criteria and extract them

# create directory to store the frames
if not os.path.isdir('frames'):
    os.makedirs('frames')

# extract and save the frames
emda.export_frames_by_analysis(variant=protein_ID, replica='R1', analysis_name='combined_criteria', folder='./frames', out_name='*')
#print(list(emda.analyses['combined_criteria'].result.values())[0]['R1'])


#total_length = 1000 * len(traj_1)
#print(f'total length:\t{total_length}')
#selected_frames = frame_extractor.frame_selector(criteria, bins=10, frames_per_bin=10)
#frame_extractor.trajectory_frame_extractor(emda.universe[protein_ID]['R1'], selected_frames, folder='./') 

/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/sergiortizropero/miniconda3/envs/md_traj/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/Us